In [2]:
%load_ext autotime
import pandas as pd
import requests
from datetime import datetime
from tqdm import tqdm
import os

time: 3.95 s (started: 2023-01-22 21:04:17 -05:00)


In [3]:
D_NA_DESC = pd.read_excel('API Resources/2022_NAICS_Structure.xlsx',skiprows=2)
D_NA_DESC.loc[D_NA_DESC['2022 NAICS Code'].astype(str).apply(len)==6,'Code Type'] = 'NA6'
D_NA_DESC.loc[D_NA_DESC['2022 NAICS Code'].astype(str).apply(len)==5,'Code Type'] = 'NA5'
D_NA_DESC.loc[D_NA_DESC['2022 NAICS Code'].astype(str).apply(len)==4,'Code Type'] = 'NA4'
D_NA_DESC.loc[D_NA_DESC['2022 NAICS Code'].astype(str).apply(len)==3,'Code Type'] = 'NA3'
D_NA_DESC.loc[D_NA_DESC['2022 NAICS Code'].astype(str).apply(len)==2,'Code Type'] = 'NA2'
D_NA_DESC.isna().sum()
D_NA_DESC.drop('Change Indicator',axis=1,inplace=True)
D_NA_DESC[D_NA_DESC['Code Type']=='NA2'].shape
D_NA_DESC[D_NA_DESC['Code Type']=='NA3'].shape
D_NA_DESC[D_NA_DESC['Code Type']=='NA4'].shape
D_NA_DESC[D_NA_DESC['Code Type']=='NA5'].shape
# D_NA_DESC[D_NA_DESC['Code Type']=='NA6'].shape
D_NA_DESC = D_NA_DESC.dropna()

for intE in range(2,6):D_NA_DESC[f'NA{intE}'] = D_NA_DESC[f'2022 NAICS Code'].apply(lambda x:str(x)[:intE])

D_NA_DESC = D_NA_DESC[D_NA_DESC['Code Type']=='NA6']\
    .merge(D_NA_DESC[D_NA_DESC['Code Type']=='NA5'][['NA5','2022 NAICS Title']].rename({"2022 NAICS Title":'NA5 Description'},axis=1)
    ,how='left',suffixes=["_NA5"])\
        .merge(D_NA_DESC[D_NA_DESC['Code Type']=='NA4'][['NA4','2022 NAICS Title']].rename({"2022 NAICS Title":'NA4 Description'},axis=1)
    ,how='left',suffixes=["_NA4"])\
        .merge(D_NA_DESC[D_NA_DESC['Code Type']=='NA3'][['NA3','2022 NAICS Title']].rename({"2022 NAICS Title":'NA3 Description'},axis=1)
    ,how='left',suffixes=["_NA3"])\
        .merge(D_NA_DESC[D_NA_DESC['Code Type']=='NA2'][['NA2','2022 NAICS Title']].rename({"2022 NAICS Title":'NA2 Description'},axis=1)
    ,how='left',suffixes=["_NA2"])

time: 1.82 s (started: 2023-01-22 21:04:21 -05:00)


In [4]:
D_NA_DESC = D_NA_DESC.rename({'2022 NAICS Code':'NAICS'},axis=1)#.pivot_table(index=['2022 NAICS Code','NA2','NA3','NA4','NA5'],columns='Code Type',aggfunc=lambda x:' '.join(x)).to_clipboard()
D_NA_DESC['NAICS'] = D_NA_DESC['NAICS'].astype(str)

time: 2.9 ms (started: 2023-01-22 21:04:22 -05:00)


In [5]:
D_NA_DESC.head()

,NAICS,2022 NAICS Title,Code Type,NA2,NA3,NA4,NA5,NA5 Description,NA4 Description,NA3 Description,NA2 Description
0,111110,Soybean Farming,NA6,11,111,1111,11111,Soybean FarmingT,Oilseed and Grain FarmingT,Crop ProductionT,"Agriculture, Forestry, Fishing and HuntingT"
1,111120,Oilseed (except Soybean) Farming,NA6,11,111,1111,11112,Oilseed (except Soybean) FarmingT,Oilseed and Grain FarmingT,Crop ProductionT,"Agriculture, Forestry, Fishing and HuntingT"
2,111130,Dry Pea and Bean Farming,NA6,11,111,1111,11113,Dry Pea and Bean FarmingT,Oilseed and Grain FarmingT,Crop ProductionT,"Agriculture, Forestry, Fishing and HuntingT"
3,111140,Wheat Farming,NA6,11,111,1111,11114,Wheat FarmingT,Oilseed and Grain FarmingT,Crop ProductionT,"Agriculture, Forestry, Fishing and HuntingT"
4,111150,Corn Farming,NA6,11,111,1111,11115,Corn FarmingT,Oilseed and Grain FarmingT,Crop ProductionT,"Agriculture, Forestry, Fishing and HuntingT"


time: 177 ms (started: 2023-01-22 21:04:23 -05:00)


In [6]:
# [31,32,33,44,45,48,49]
D_NA_DESC.loc[D_NA_DESC['NA2']=='31','NA2 Description'] = 'Manufacturing 1'
D_NA_DESC.loc[D_NA_DESC['NA2']=='32','NA2 Description'] = 'Manufacturing 2'
D_NA_DESC.loc[D_NA_DESC['NA2']=='33','NA2 Description'] = 'Manufacturing 3'
D_NA_DESC.loc[D_NA_DESC['NA2']=='44','NA2 Description'] = 'Retail Trade 1'
D_NA_DESC.loc[D_NA_DESC['NA2']=='45','NA2 Description'] = 'Retail Trade 2'
D_NA_DESC.loc[D_NA_DESC['NA2']=='48','NA2 Description'] = 'Transportation and Warehousing 1'
D_NA_DESC.loc[D_NA_DESC['NA2']=='49','NA2 Description'] = 'Transportation and Warehousing 2'



time: 558 ms (started: 2023-01-22 21:04:23 -05:00)


In [7]:
D_NA_DESC[['NA2','NA2 Description']].drop_duplicates()

,NA2,NA2 Description
0,11,"Agriculture, Forestry, Fishing and HuntingT"
64,21,"Mining, Quarrying, and Oil and Gas ExtractionT"
85,22,UtilitiesT
99,23,ConstructionT
130,31,Manufacturing 1
199,32,Manufacturing 2
295,33,Manufacturing 3
476,42,Wholesale TradeT
545,44,Retail Trade 1
573,45,Retail Trade 2


time: 235 ms (started: 2023-01-22 21:04:23 -05:00)


In [8]:
'98', '99', '21', '93', '91'

('98', '99', '21', '93', '91')

time: 265 ms (started: 2023-01-22 21:04:24 -05:00)


In [9]:
# hslistDf = []
na2listDf = []
na3listDf = []
# for fileName in tqdm(os.listdir('.apiResponses/hs')):
#     if 'HSCYDT_2022' in fileName:hslistDf.append(pd.read_csv(os.path.join('.apiResponses/hs',fileName),compression='gzip'))
for fileName in tqdm(os.listdir('.apiResponses/naics')):
    # if 'NA2_NACYDT_2022' in fileName:na2listDf.append(pd.read_csv(os.path.join('.apiResponses/naics',fileName),compression='gzip'))
    if 'NA6_NACYDT_202' in fileName:
        
        temp = pd.read_csv(os.path.join('.apiResponses/naics',fileName),compression='gzip',low_memory=False)
        temp = temp[temp['CTY_NAME']=='INDIA']
        na3listDf.append(temp)

# hscydtDf = cleaningResponses(pd.concat(hslistDf))
na6Df = pd.concat(na3listDf)
na6Df['NA2'] = na6Df['NAICS'].apply(lambda x:str(x)[:2])
na6Df = na6Df.merge(na6Df,how='left')
na6Df = na6Df[na6Df['CTY_NAME']=='INDIA']

na6Df.loc[na6Df['NA2']=='31','NA2 Description'] = 'Manufacturing 1'
na6Df.loc[na6Df['NA2']=='32','NA2 Description'] = 'Manufacturing 2'
na6Df.loc[na6Df['NA2']=='33','NA2 Description'] = 'Manufacturing 3'
na6Df.loc[na6Df['NA2']=='44','NA2 Description'] = 'Retail Trade 1'
na6Df.loc[na6Df['NA2']=='45','NA2 Description'] = 'Retail Trade 2'
na6Df.loc[na6Df['NA2']=='48','NA2 Description'] = 'Transportation and Warehousing 1'
na6Df.loc[na6Df['NA2']=='49','NA2 Description'] = 'Transportation and Warehousing 2'
na6Df.loc[na6Df['NA2']=='11','NA2 Description'] = 'Agriculture, Forestry, Fishing and HuntingT'
na6Df.loc[na6Df['NA2']=='21','NA2 Description'] = 'Mining, Quarrying, and Oil and Gas ExtractionT'
na6Df.loc[na6Df['NA2']=='91','NA2 Description'] = 'Waste and Scrap'
na6Df.loc[na6Df['NA2']=='93','NA2 Description'] = 'Used or Second-Hand Merchandise'
na6Df.loc[na6Df['NA2']=='98','NA2 Description'] = 'Goods Returned (EXPORTS FOR CANADA ONLY)'
na6Df.loc[na6Df['NA2']=='99','NA2 Description'] = 'Other Speical Classification Provisions'
na6Df

100%|██████████| 591/591 [01:05<00:00,  9.02it/s] 


,NAICS,NAICS_LDESC,NAICS_SDESC,CTY_CODE,CTY_NAME,DISTRICT,DIST_NAME,DUT_VAL_MO,CON_VAL_MO,CAL_DUT_MO,...,VES_CHA_MO,VES_VAL_MO,VES_WGT_MO,YEAR,MONTH,COMM_LVL,SUMMARY_LVL2,YearMonth,NA2,NA2 Description
0,111110,SOYBEANS,SOYBEANS,5330,INDIA,13,"BALTIMORE, MD",0,769300,0,...,112700,769300,660200,2022,1,NA6,NACYDT,2022-01,11,"Agriculture, Forestry, Fishing and HuntingT"
1,111120,OILSEEDS (EXCEPT SOYBEAN),OILSEEDS (EXCEPT SOYBEAN),5330,INDIA,4,"BOSTON, MA",0,4748,0,...,710,4748,1212,2022,1,NA6,NACYDT,2022-01,11,"Agriculture, Forestry, Fishing and HuntingT"
2,111120,OILSEEDS (EXCEPT SOYBEAN),OILSEEDS (EXCEPT SOYBEAN),5330,INDIA,7,"OGDENSBURG, NY",0,29512,0,...,0,0,0,2022,1,NA6,NACYDT,2022-01,11,"Agriculture, Forestry, Fishing and HuntingT"
3,111120,OILSEEDS (EXCEPT SOYBEAN),OILSEEDS (EXCEPT SOYBEAN),5330,INDIA,9,"BUFFALO, NY",0,22078,0,...,0,0,0,2022,1,NA6,NACYDT,2022-01,11,"Agriculture, Forestry, Fishing and HuntingT"
4,111120,OILSEEDS (EXCEPT SOYBEAN),OILSEEDS (EXCEPT SOYBEAN),5330,INDIA,10,"NEW YORK CITY, NY",78683,1409019,147,...,154339,1406908,719305,2022,1,NA6,NACYDT,2022-01,11,"Agriculture, Forestry, Fishing and HuntingT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225482,990000,OTHER SPECIAL CLASSIFICATION PROVISIONS,OTHER SPECIAL CLASSIFICATION PROVISIONS,5330,INDIA,41,"CLEVELAND, OH",9718,29816,136,...,1937,4124,186,2021,12,NA6,NACYDT,2021-12,99,Other Speical Classification Provisions
225483,990000,OTHER SPECIAL CLASSIFICATION PROVISIONS,OTHER SPECIAL CLASSIFICATION PROVISIONS,5330,INDIA,45,"ST. LOUIS, MO",0,103464,0,...,26795,103464,18192,2021,12,NA6,NACYDT,2021-12,99,Other Speical Classification Provisions
225484,990000,OTHER SPECIAL CLASSIFICATION PROVISIONS,OTHER SPECIAL CLASSIFICATION PROVISIONS,5330,INDIA,49,"SAN JUAN, PR",0,0,0,...,0,0,0,2021,12,NA6,NACYDT,2021-12,99,Other Speical Classification Provisions
225485,990000,OTHER SPECIAL CLASSIFICATION PROVISIONS,OTHER SPECIAL CLASSIFICATION PROVISIONS,5330,INDIA,52,"MIAMI, FL",0,0,0,...,0,0,0,2021,12,NA6,NACYDT,2021-12,99,Other Speical Classification Provisions


time: 1min 7s (started: 2023-01-22 21:04:25 -05:00)


In [10]:
tempdf = na6Df.fillna('UKN').groupby(['YearMonth','NA2',"NAICS",'NA2 Description',"NAICS_LDESC"])[["GEN_CHA_MO","GEN_VAL_MO","GEN_CIF_MO",'AIR_VAL_MO','VES_VAL_MO','CC_MO']].sum().reset_index()
tempdf['NAICS'] = tempdf['NAICS'].astype(str)
tempdf = tempdf.sort_values('GEN_CIF_MO',ascending=False)
tempdf

,YearMonth,NA2,NAICS,NA2 Description,NAICS_LDESC,GEN_CHA_MO,GEN_VAL_MO,GEN_CIF_MO,AIR_VAL_MO,VES_VAL_MO,CC_MO
7642,2021-10,33,339910,Manufacturing 3,JEWELRY AND SILVERWARE,31754138,1718386081,1750140219,1700780629,16721635,24461
6918,2021-08,33,339910,Manufacturing 3,JEWELRY AND SILVERWARE,30126220,1633735717,1663861937,1619364174,12131151,23497
11871,2022-10,33,339910,Manufacturing 3,JEWELRY AND SILVERWARE,23849081,1563615961,1587465042,1553506631,9409386,19981
6558,2021-07,33,339910,Manufacturing 3,JEWELRY AND SILVERWARE,25477489,1537636716,1563114205,1525820144,11569383,21525
10445,2022-06,33,339910,Manufacturing 3,JEWELRY AND SILVERWARE,25394691,1519759304,1545153995,1502688769,14387016,19984
...,...,...,...,...,...,...,...,...,...,...,...
1631,2020-05,33,336112,Manufacturing 3,LIGHT TRUCK AND UTILITY VEHICLE,0,0,0,0,0,0
1632,2020-05,33,336211,Manufacturing 3,MOTOR VEHICLE BODIES,0,0,0,0,0,0
6961,2021-09,21,212299,"Mining, Quarrying, and Oil and Gas ExtractionT",ALL OTHER METAL ORES,0,0,0,0,0,0
2758,2020-09,21,212324,"Mining, Quarrying, and Oil and Gas ExtractionT",KAOLIN AND OTHER KAOLINIC CLAYS,0,0,0,0,0,0


time: 626 ms (started: 2023-01-22 21:05:44 -05:00)


In [36]:
NA2Summary = tempdf.groupby(['NA2','NA2 Description'])[['GEN_CHA_MO','GEN_VAL_MO','CC_MO']].sum().sort_values('GEN_VAL_MO',ascending=False).reset_index()

time: 12.1 ms (started: 2023-01-22 20:33:13 -05:00)


In [13]:
tempdf.groupby('NAICS_LDESC')[['GEN_CHA_MO','GEN_VAL_MO']].sum().sort_values('GEN_VAL_MO',ascending=False).reset_index()

,NAICS_LDESC,GEN_CHA_MO,GEN_VAL_MO
0,JEWELRY AND SILVERWARE,269921042,15124629912
1,PHARMACEUTICAL PREPARATIONS,240469476,8314333560
2,PETROLEUM REFINERY PRODUCTS,136723696,3491837991
3,ALL OTHER BASIC ORGANIC CHEMICALS,160228860,3413218295
4,"WOMEN'S, GIRLS', AND INFANTS' CUT AND SEW APPAREL",161911574,3239106694
...,...,...,...
354,IRON ORES,365,9216
355,GOATS AND OTHER FINE ANIMAL HAIR,410,8543
356,HORSES AND OTHER EQUINE,529,7130
357,ICE,31,4524


time: 161 ms (started: 2023-01-22 20:22:32 -05:00)


array(['33', '32', '31', '11', '98', '99', '93', '21', '91'], dtype=object)

time: 5.02 ms (started: 2023-01-22 21:31:40 -05:00)


In [53]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# fig = make_subplots(specs=[[{"secondary_y": True}]])
def MoMChangePlotbyNA2(tempdf,NA2Code):
    totalCIF33 = tempdf[tempdf['NA2']==NA2Code].groupby('YearMonth')[['AIR_VAL_MO','VES_VAL_MO']].sum().reset_index()
    
    fig = go.Figure(data=[
        go.Bar(name='by Air', 
               x = totalCIF33['YearMonth'], 
               y = totalCIF33['AIR_VAL_MO']/1e9,
               yaxis='y1',
               hoverinfo='text',
            #    hover=NA2Summary['NA2 Description'],
               text = "Air:"+(totalCIF33['AIR_VAL_MO']/1e9).round(2).astype(str) + 'B',
               textposition="inside",
               insidetextanchor="end"),
    
        go.Bar(name='by Vessel',
               x=totalCIF33['YearMonth'], 
               y=totalCIF33['VES_VAL_MO']/1e9,
               yaxis='y1',
               hoverinfo='text',
               text="Vessel:"+(totalCIF33['VES_VAL_MO']/1e9).round(2).astype(str) + 'B',
               textposition="inside",
            #    texttemplate='%text',
               textfont=dict(
            family="sans serif",
            size=24,
            color="white"
        )),
    ])
    
    fig.add_trace(go.Scatter(
        x=totalCIF33['YearMonth'],
        y=(totalCIF33['VES_VAL_MO']+totalCIF33['AIR_VAL_MO']).pct_change()*100,
        # xperiod="M1",
        yaxis="y2",
        name="MoM Growth",
        hoverinfo='text',
        text="{y2:.2f}%",
        xperiodalignment="middle",
        hovertemplate="%{y:.2f}%%{_xother}"
    ))
    fig.update_traces(textfont_size=30)
    fig.update_layout(autosize=False,
        yaxis1=dict(side='left'),
        yaxis2=dict(title='MoM Growth',overlaying= 'y',side='right'),
        legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1),
        width=1400,
        height=500,
        margin=dict(l=50, r=50, b=100, t=50, pad=4),
        barmode='group',
        title=f"Inbound CIF by Air and Vessel for {tempdf[tempdf['NA2']==NA2Code]['NA2 Description'].unique()[0]} in US From India",
        xaxis_title="Year-Month",
        yaxis_title="Dollar value (in Billions)")
    # fig.show()
    
    fig.update_layout(hovermode="x unified")
    fig.show()

time: 2.95 ms (started: 2023-01-22 21:32:01 -05:00)


In [55]:
for na2Code in tempdf['NA2'].unique():MoMChangePlotbyNA2(tempdf,na2Code)

time: 421 ms (started: 2023-01-22 21:33:16 -05:00)


In [43]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
# pio.renderers.default = "browser"

fig = go.Figure(data=[
    go.Bar(name='Total General Imports Value', 
           x = NA2Summary['NA2'], 
           y = NA2Summary['GEN_VAL_MO']/1e9,
        #    hover=NA2Summary['NA2 Description'],
           text = NA2Summary['NA2 Description'] +": " + (NA2Summary['GEN_VAL_MO']/1e9).round(2).astype(str) + 'B',
           textposition="inside",
           insidetextanchor="start"),

    go.Bar(name='Total Charges',
           x=NA2Summary['NA2'], 
           y=NA2Summary['GEN_CHA_MO']/1e9,
           text=(NA2Summary['GEN_CHA_MO']/1e9).round(2).astype(str) + 'B',
           textposition="inside",
        #    texttemplate='%text',
           textfont=dict(
        family="sans serif",
        size=24,
        color="white"
    )),
])
# fig.add_trace(
#     go.Scatter(
#         x=[str(tup) for tup in zip(top5_case2["OP_CARRIER"],top5_case2["ORIGIN"],top5_case2["DESTINATION"])],
#         y=top5_case2["profit"],
#         mode="markers+text",
#         name="Gross Profit",
#         text=top5_case2["profit"],
#         textposition="bottom center",
#         texttemplate='%{text:.4s}',
#         textfont=dict(family="sans serif",
#                       size=20,
#                       color="black"),
#         marker=dict(color='LightSkyBlue',
#                     size=10,
#                     line=dict(
#                         color='MediumPurple',
#                         width=1)
#                     )
#     ))
# Change the bar mode
fig.update_traces(textfont_size=16)
fig.update_layout(autosize=False,legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1),
    width=1400,
    height=800,
    margin=dict(l=50, r=50, b=100, t=50, pad=4),
    barmode='stack',
    title="Top Imports and Charges by NAICS2 in Dollar Value",
    xaxis_title="NAICS2 Descriptions",
    yaxis_title="Dollar value (in Billions)")
fig.show()

time: 28.4 ms (started: 2023-01-22 20:41:52 -05:00)


In [ ]:
na2dtDf = cleaningResponses(pd.concat(na2listDf))
na2Summary = na2dtDf.groupby(['NAICS','NAICS_SDESC'])[valFields].sum()
na2Summary.to_clipboard()

time: 203 ms (started: 2023-01-21 23:23:32 -05:00)


In [ ]:
na3dtDf.columns

Index(['NAICS', 'NAICS_LDESC', 'NAICS_SDESC', 'CTY_CODE', 'CTY_NAME',
       'DISTRICT', 'DIST_NAME', 'DUT_VAL_MO', 'CON_VAL_MO', 'CAL_DUT_MO',
       'GEN_CHA_MO', 'GEN_VAL_MO', 'GEN_CIF_MO', 'CC_MO', 'CNT_CHA_MO',
       'CNT_VAL_MO', 'CNT_WGT_MO', 'AIR_CHA_MO', 'AIR_VAL_MO', 'AIR_WGT_MO',
       'VES_CHA_MO', 'VES_VAL_MO', 'VES_WGT_MO', 'YEAR', 'MONTH', 'COMM_LVL',
       'SUMMARY_LVL2', 'YearMonth'],
      dtype='object')

time: 4.39 ms (started: 2023-01-21 22:24:14 -05:00)


In [ ]:
na3dtDf = pd.concat(na3listDf)
valFields = ['GEN_VAL_MO','DUT_VAL_MO','CAL_DUT_MO','GEN_CHA_MO','CON_VAL_MO','AIR_CHA_MO','AIR_VAL_MO','CNT_CHA_MO','CNT_VAL_MO','VES_CHA_MO','VES_VAL_MO','GEN_CIF_MO']
na3dtDf[valFields] = na3dtDf[valFields].astype(int)/1e6
na3Summary = na3dtDf.groupby(['NAICS','NAICS_SDESC'])[valFields].sum().reset_index()
na3Summary = na3Summary.sort_values('CON_VAL_MO',ascending=False)
na3Summary.to_csv("datasets/importsConsumptionAIR_CNT.csv",index=False)
na3Summary['NAICS_2'] = na3Summary['NAICS'].apply(lambda x:str(x)[:2])
na3Summary['totals'] = na3Summary[['AIR_VAL_MO','VES_VAL_MO']].sum(axis=1)
na3Summary['diff'] = na3Summary['GEN_CIF_MO']-na3Summary[['GEN_VAL_MO','GEN_CHA_MO']].sum(axis=1)
na3Summary.to_clipboard(index=False)

time: 2.12 s (started: 2023-01-21 23:07:15 -05:00)


In [ ]:
import plotly.express as px
fig = px.bar(na3Summary, x="NAICS_SDESC", y=['AIR_VAL_MO','CNT_VAL_MO'], 
                        title="Wide-Form Input",
                        width=1750,height=600)
fig.show()

time: 88.8 ms (started: 2023-01-21 22:01:28 -05:00)
